<a href="https://colab.research.google.com/github/elenancalima/Troph_Min_5to2/blob/main/Minimal_5_to_2_backbone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

PROJ_DIR = "/content/drive/MyDrive/Troph_Min_5to_2_mod"
import os, textwrap, pathlib
os.makedirs(PROJ_DIR, exist_ok=True)
print("Project dir:", PROJ_DIR)


Mounted at /content/drive
Project dir: /content/drive/MyDrive/Troph_Min_5to_2_mod


In [2]:
import sys, importlib, os, torch

# --- project path ---
PROJ_DIR = "/content/drive/MyDrive/Troph_Min_5to_2_mod"   # <-- set this
if PROJ_DIR not in sys.path: sys.path.insert(0, PROJ_DIR)

# --- reload core modules ---
import config;          importlib.reload(config)
import fake_data;       importlib.reload(fake_data)   # writes fake data to disk (per config subfolders)
import folder_dataset;  importlib.reload(folder_dataset)    # disk-backed dataset
import data_factory;    importlib.reload(data_factory)      # loaders from folders
import model_factory;   importlib.reload(model_factory)     # build_stage1/2
import trainer;         importlib.reload(trainer)           # train_stages/save_weights
import pipeline;        importlib.reload(pipeline)          # SingleFramePipeline
import troph;        importlib.reload(troph)          # SingleFramePipeline


<module 'troph' from '/content/drive/MyDrive/Troph_Min_5to_2_mod/troph.py'>

In [4]:
# Generate FAKE on disk → build loaders → train (minimal)
TRAIN_ROOT = "/content/fake_train_v1"
VAL_ROOT   = "/content/fake_val_v1"

# 1) materialize fake data on disk (per config subfolders)
fake_data.make_fake_trainval_folders(TRAIN_ROOT, VAL_ROOT, n_train=64, n_val=32, seed=0)


In [5]:
#@title Minimal: NEW model → train → predict (Ultralytics-like)
import sys, importlib
PROJ_DIR = "/content/drive/MyDrive/Troph_Min_5to_2_mod"
if PROJ_DIR not in sys.path: sys.path.insert(0, PROJ_DIR)

import troph;          importlib.reload(troph)
from troph import Troph

# Reuse the paths you already created
# TRAIN_ROOT = ".../fake_train_v1"
# VAL_ROOT   = ".../fake_val_v1"

# model = Troph(weights_dir=f"{PROJ_DIR}/weights")  # ← resume from saved weights (commented out)
model = Troph()                                      # ← default: start fresh

model.train(train_root=TRAIN_ROOT, val_root=VAL_ROOT, epochs=3)   # train
model.save(f"{PROJ_DIR}/weights")                                 # save
summary = model.predict(root=VAL_ROOT)                             # infer & save PNGs
print(summary)


epoch 1: val_loss=0.3963
epoch 2: val_loss=0.1129
epoch 3: val_loss=0.0577
[predict] processed 50 frames...
[predict] done: 64 frames.
{'processed_frames': 64, 'out_troph_dir': '/content/fake_val_v1/pred_troph_point', 'out_part_dir': '/content/fake_val_v1/pred_participant_line'}


In [7]:
from troph import Troph
print("import done")

VIDEO_ROOT = "/content/drive/MyDrive/MainConnection_VidRoots/FoodLimit2.5D_gR0033_feeding"
model = Troph(weights_dir=f"{PROJ_DIR}/weights")

# Tip: batch_size>1 amortizes GPU↔CPU transfers; num_workers=0 avoids Drive thrash
print(model.predict(
    root=VIDEO_ROOT,
    batch_size=8,
    num_workers=0,
    clear_outputs=True,
    progress_every=50,
    use_local_tmp=True,             # write to /content and commit to Drive after
    tmp_root="/content/troph_tmp_preds"
))


KeyboardInterrupt: 

Resume from here

Resume from here.

Now, dummy training.


BELOW SETUP IS UNTESTED

ACTUAL RUN STARTS HERE

This takes considerably longer than YOLO inference, beware.